In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from matplotlib import pyplot as plt

In [2]:
model = load_model('model-004.model')

In [3]:
# label dictionary for each image class and color dictionary for bounding boxes
labels_dict = {0:'correctly_worn', 1:'incorrectly_worn', 2:'no_mask'}
color_dict = {0:(0,255,0), 1:(0,255,255), 2:(0,0,255)}

In [4]:
# import face detection program

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [5]:
size = 2
webcam = cv2.VideoCapture(0)

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1)
    
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))
    
    faces = classifier.detectMultiScale(mini)
    
    for f in faces:
        (x, y, w, h) = [v * size for v in f]
        face_img = im[y-40:y+h+40, x-40:x+w+40]
        resized = cv2.resize(face_img, (224,224))
        normalized = resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped= np.vstack([reshaped])
        result = model.predict(reshaped)
        print(result)
        
        label = np.argmax(result,axis=1)[0]
        
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im,labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('LIVE', im)
    key = cv2.waitKey(10)
    if key == 27: # ESC key
        break
        
webcam.release()

cv2.destroyAllWindows()

[[0.7308862  0.09533525 0.17377847]]
[[0.81097144 0.0809999  0.10802866]]
[[5.0045160e-04 4.9892287e-03 9.9451029e-01]]
[[4.1189008e-09 8.8310653e-06 9.9999118e-01]]
[[0.0012771  0.00745779 0.9912651 ]]
[[0.00158382 0.00980223 0.98861396]]
[[3.8208686e-08 2.5556956e-05 9.9997437e-01]]
[[4.9388607e-04 6.8355077e-03 9.9267066e-01]]
[[4.6148930e-08 3.0875868e-05 9.9996912e-01]]
[[9.2919059e-09 1.2032003e-05 9.9998796e-01]]
[[7.7252684e-04 8.4703062e-03 9.9075717e-01]]
[[1.2041523e-09 3.0301603e-06 9.9999702e-01]]
[[7.3290564e-04 7.4054427e-03 9.9186158e-01]]
[[9.1905605e-10 3.4941238e-06 9.9999654e-01]]
[[0.00220073 0.01008687 0.98771244]]
[[3.2424476e-09 7.0559299e-06 9.9999297e-01]]
[[0.00490615 0.01508833 0.9800055 ]]
[[5.6001728e-09 8.6563850e-06 9.9999130e-01]]
[[0.00232037 0.01122622 0.98645335]]
[[2.1510318e-09 4.7676758e-06 9.9999523e-01]]
[[0.0020672  0.00965966 0.9882732 ]]
[[8.6074914e-10 3.2598218e-06 9.9999678e-01]]
[[7.9270895e-04 6.7364485e-03 9.9247086e-01]]
[[5.6340604e-1

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
faces

In [ ]:
plt.imshow(face_img, interpolation = 'nearest')
plt.show()

In [ ]:
plt.imshow(mini, interpolation = 'nearest')
plt.show()